This notebook will produce a graph as follows:
- X axis is
    - em-var status (categorical)
    - mean predicted activity bin
- Y-axis is the odds ratio of phylop significance representation in that category

In [1]:
import glob
import pandas as pd

In [2]:
#importlib code just for debugging : to allow changes in scripts we are imporing from
#reflect here. It can be removed if you are not making changes to the scripts we are imporing from

import importlib ##can remove
import graphing_config as gc
import or_common as orc
importlib.reload(gc) ##can remove
importlib.reload(orc) ##can remove

<module 'or_common' from '/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_scripts/5.graphs/odds_ratio/or_common.py'>

In [7]:
#phylop_emvar=pd.read_csv(glob.glob(GC.data_base_path+"phylop_emvar/*.csv")[0])
master_table=pd.read_csv(glob.glob("/home/mcn26/varef/scripts/noon_data/4.old.count/count_all.csv/*.csv")[0])

In [8]:
master_table

,CADD>=10,CADD>=20,CADD>=30,CADD>=40,CADD>=50,is_in_dELS,is_in_CA,is_in_pELS,is_in_CA-H3K4me3,is_in_CA-CTCF,...,SKNSH__skew_(1&5^ Inf),SKNSH__ref_(-Inf^-1),SKNSH__ref_[-1^0),SKNSH__ref_[0^1),SKNSH__ref_[1^2),SKNSH__ref_[2^3),SKNSH__ref_[3^4),SKNSH__ref_[4^5),SKNSH__ref_[5^Inf),count
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,84
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,2123
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,149
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,578
4,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,1930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360364,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,1
3360365,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,1
3360366,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,1
3360367,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,1


In [9]:
#extract some relevant sets of columns & drop all the columns we don't need...

columns=master_table.columns.to_list()

genomic_regions_cols=[i for i in columns if i.startswith("is_in_") ]
mean_activity_cols=[i for i in columns if i.startswith("mean_skew_")]
emVar_cols=[i for i in columns if i.startswith("emVar_")]



In [10]:
#perform the actual subsetting
#columns_to_keep is columns we would want no matter what
columns_to_keep=genomic_regions_cols+mean_activity_cols+["count","phylop_significant"]

master_table=master_table[columns_to_keep+emVar_cols+["pleio"]]

In [13]:
#break the table into two sub-tables : one for pleiotropy analysis, and one for emvar status analysis

pleio_table=master_table[columns_to_keep+["pleio"]].copy()
emvar_table=master_table[columns_to_keep+emVar_cols].copy()

#delete the original table to free up memory
del master_table

In [14]:
#remove rows we don't need. Rows which are identical except for their counts
#were distinct rows because of columns we have discarded above
#so this summing finishes the above subsetting. 

def cull_rows(table,columns):
    return table.groupby(columns).sum().reset_index(inplace=True)



#master_table=master_table.groupby(["phylop_significant"]+emVar_cols+mean_activity_cols+genomic_regions_cols).sum()
#master_table.reset_index(inplace=True)

NameError: name 'master_table' is not defined

In [20]:
#fix the dumb character substitutions
#(pyspark hates certain characters, so we had to put substitutes)
#both in the actual table...
master_table.columns=[col_name.replace("&", ".").replace("^",",") for col_name in master_table.columns]
#...and in the column names we extracted above. 
mean_activity_cols=[i.replace("&", ".").replace("^",",") for i in mean_activity_cols]

In [11]:
emvar_or_pleio="emVar"

In [7]:
#select one combo of subset cata
selected=master_table[master_table["is_in_PLS"]]
#drop now irrelevant cata columns
selected=selected.drop(genomic_regions_cols,axis=1)
#break into multiple tables
dic={k:selected[selected[k]] for k in mean_activity_cols}

mean_activity_strata_collector={}
for mean_activity_strata in dic.keys():
    
    emvar_collector={}
    
    
    inner_cols=[]
    if emvar_or_pleio=="emVar":
        inner_cols=emVar_cols
    elif emvar_or_pleio=="pleio":
        inner_cols=...
    else:
        print("Error : invalid emvar / pleio setting")
        exit(-1)
        
    
    for emVar in inner_cols:
    
        emvar_collector[emVar]=orc.compute_OR(df=dic[mean_activity_strata],
                      a=emVar,
                      a_val=True,
                      b="phylop_significant",
                      b_val=True)
        
        #break
    mean_activity_strata_collector[mean_activity_strata]=emvar_collector



/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:156: RuntimeWarning: divide by zero encountered in divide
  se = np.sqrt((1/table).sum())
/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:168: RuntimeWarning: invalid value encountered in scalar subtract
  loglow = log_or - z*se
/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:155: RuntimeWarning: divide by zero encountered in log
  log_or = np.log(oddsratio)
/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:169: RuntimeWarning: invalid value encountered in scalar add
  loghigh = log_or + z*se


In [12]:
#Let's flatten the nested dictionary we just created. 
records=[]
for skew, inner in mean_activity_strata_collector.items():
    for cata,stats in inner.items():
        record={"mean_skew":skew,emvar_or_pleio:cata}
        record.update(stats)
        records.append(record)

records=pd.DataFrame(records)

display(records)

,mean_skew,emVar,OR,ci_lower,ci_upper,p
0,mean_skew_(-Inf^ -1&5),emVar_K562,4.549033,1.558754,13.275799,2.105654e-03
1,mean_skew_(-Inf^ -1&5),emVar_SKNSH,inf,NaN,inf,1.000000e+00
2,mean_skew_(-Inf^ -1&5),emVar_HepG2,NaN,0.000000,inf,1.000000e+00
3,mean_skew_[-1&5^ -1&0),emVar_K562,2.947867,1.984914,4.377981,5.508470e-09
4,mean_skew_[-1&5^ -1&0),emVar_SKNSH,2.196707,1.580554,3.053059,1.191561e-06
5,mean_skew_[-1&5^ -1&0),emVar_HepG2,1.800485,1.039273,3.119244,3.866178e-02
6,mean_skew_[-1&0^ -0&5),emVar_K562,1.414167,1.339351,1.493162,4.669224e-37
7,mean_skew_[-1&0^ -0&5),emVar_SKNSH,1.265541,1.211696,1.321778,1.073724e-26
8,mean_skew_[-1&0^ -0&5),emVar_HepG2,1.133626,1.092070,1.176764,4.542361e-11
9,mean_skew_[-0&5^ -0&2),emVar_K562,1.287225,1.249381,1.326216,2.501501e-60
